In [1]:
import ipywidgets as widgets
from IPython.display import display
from dotenv import load_dotenv
import os
import sys
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
sys.path.append(os.path.abspath(".."))
from ift6758.GamesUtils import GamesUtils

In [2]:
class SimpleWidget:
    def __init__(self):
        """Interactive Widget for exploring the play-by-play games data

        Attributs:
            data_path : String
                Path to the directory containing game JSON data.

            game_type : dict
                Mapping of the game types to their API codes ('regular season' -> '02').

            rink_image_path : String
                Path to the rink image.

            roaster : dict
                Mapping of player IDs to player info (name, headshot).

            team : dict
                Mapping of team IDs to team name and home/away.

            game_id_slider : widgets.SelectionSlider
                Slider for selecting the current game ID.

            game_info_box : widgets.HTML
                Widget for displaying the current game information.

            event_info_box : widgets.HTML
                Widget for displaying detailed info for a selected event.

            event_id_slider : widgets.SelectionSlider
                Slider for selecting the event ID of a game.

            season_drop_down_menu : widgets.Dropdown
                Dropdown for selecting the season.

            game_type_radio_button : widgets.RadioButtons
                Radio buttons for selecting the type of game (preseason, regular season, playoff).

            rink_output : widgets.Output
                Output widget to render the rink image and event placement.

            ui : widgets.VBox
                Container holding all widget elements for display.
        """
        self.data_path = '../games_data'

        self.game_type = {'preseason' : '01', 'regular season':'02', 'playoff':'03'}
        self.rink_image_path = '../figures/nhl_rink.png'
        self.roaster = {}
        self.team = {}
        self.game_id_slider = widgets.SelectionSlider(
            options=[0],
            value=0,
            description="Game ID:",
            continuous_update=False
        )

        self.game_info_box = widgets.HTML(
            value="",
            placeholder="Game info",
            layout=widgets.Layout(width="500px")
        )

        self.event_info_box = widgets.HTML(
            value="",
            placeholder="Game info",
            layout=widgets.Layout(width="500px")
        )

        self.event_id_slider = widgets.SelectionSlider(
            options=[0],
            value=0,
            description="Event ID:",
            continuous_update=False
        )

        season_options = [folder for folder in os.listdir(self.data_path) if os.path.isdir(os.path.join(self.data_path, folder))]

        self.season_drop_down_menu = widgets.Dropdown(
            options = season_options,
            value=season_options[0],
            description='Season'

        )
        self.game_type_radio_button = widgets.RadioButtons(
            options=['preseason', 'regular season', 'playoff'],
            value='regular season',
            description="Type of game"
        )
        
        self.rink_output = widgets.Output()

        self.ui = widgets.VBox([self.season_drop_down_menu, 
                                self.game_type_radio_button, 
                                self.game_id_slider, 
                                self.game_info_box, 
                                self.event_id_slider, 
                                self.event_info_box, 
                                self.rink_output])

        self.season_drop_down_menu.observe(self.update_selection, names="value")
        self.game_type_radio_button.observe(self.update_selection, names="value")


        self.game_id_slider.observe(self.updates_on_game_change, names='value')

        self.event_id_slider.observe(self.update_on_event_change, names="value")
        
        self.update_selection(None)
    

        
    def update_selection(self, change):
        """Update available games when the season or game type changes. """

        season = self.season_drop_down_menu.value
        game_type = self.game_type.get(self.game_type_radio_button.value)
        season_path = os.path.join(self.data_path, season)

        game_id_list = []

        # Get all the game IDs in a season
        for file in os.listdir(season_path):
            file_path = os.path.join(season_path, file)
            if os.path.isfile(file_path) and file[4:6] == game_type:
                id = file.split('.')[0][6:]
                game_id_list.append(int(id))

        # Set the Game id slider to the all the available game id in the season
        game_id_list.sort()
        if game_id_list:
            self.game_id_slider.unobserve(self.updates_on_game_change, names='value')
            self.game_id_slider.options = game_id_list
            self.game_id_slider.value = game_id_list[0]
            self.game_id_slider.observe(self.updates_on_game_change, names='value')
            self.updates_on_game_change(None)
                
                

    def updates_on_game_change(self, change):
        """Update the widget state when a new game is selected"""

        game = self.load_game()
        if not game:
            return
        
        # Get both team player roaster
        self.built_player_roaster(game)
        self.update_game_info(game)

        event_id = list(range(len(game['plays'])))
        
        # Set the event id slider to the all the available events in a game
        event_id.sort()
        if event_id:
            self.event_id_slider.unobserve(self.update_on_event_change, names='value')
            self.event_id_slider.options = event_id
            self.event_id_slider.value = event_id[0]
            self.event_id_slider.observe(self.update_on_event_change, names='value')

            self.update_on_event_change(None)


    def update_on_event_change(self, change=None):

        """ Update the rink and event info when a new play is selected"""
        game = self.load_game()
        if not game:
            return
        
        # Draw the location of the event on the rink image
        self.draw_event_location(game)
        self.update_event_id_info(game)


    def update_game_info(self, game):
        """ Update the HTML game info box with the details of the current game.

        Args:
            game (dict): The parsed JSON dictionary containing the game play-by-play informations

        """
        #period_type = game['gameOutcome']['lastPeriodType']
        home_team = self.team.get(game['homeTeam']['id'])[0]
        away_team = self.team.get(game['awayTeam']['id'])[0]

        home_score = game['homeTeam']['score']
        away_score = game['awayTeam']['score']

        home_sog = game['homeTeam']['sog']
        away_sog = game['awayTeam']['sog']

        game_start = game['startTimeUTC']
        game_venue = game['venue']['default']

        home_logo = game['homeTeam']['logo']
        away_logo = game['awayTeam']['logo']

        html = f"""

        <div style="text-align:center; font-weight:bold;">
        <img src="{home_logo}" width="40" style="vertical-align:middle;">
            {home_team} vs {away_team}
            <img src="{away_logo}" width="40" style="vertical-align:middle;">
            <br>
            {game_venue}<br>
            {game_start}
        </div>

        <table style="text-align:center; border-collapse:collapse; width:100%">
            <tr>
                <th>{_}</th><th>Home</th><th>Away</th>
            </tr>
            <tr>
                <td>Teams</td><td>{home_team}</td><td>{away_team}</td>
            </tr>
            <tr>
                <td>Score</td><td>{home_score}</td><td>{away_score}</td>
            </tr>
            <tr>
                <td>SoG</td><td>{home_sog}</td><td>{away_sog}</td>
            </tr>
        </table>
        """
        self.game_info_box.value = html 


    def built_player_roaster(self, game):
        """Build the player roaster and team mapping for the game

        Args:
            game (dict): The parsed JSON dictionary containing the game play-by-play informations
        """
        self.roaster.clear()
        self.team.clear()
        # for player in game["rosterSpots"]:
        #     player_name = {player['playerId'] : (player['firstName']['default'] + " " + player['lastName']['default'], player['headshot'])}
        self.roaster = GamesUtils.get_game_roaster(game)
        # home_team = game['homeTeam']['abbrev'] + ' - ' + game['homeTeam']['commonName']['default']
        # away_team = game['awayTeam']['abbrev'] + ' - ' + game['awayTeam']['commonName']['default']
        # home_team_id = game['homeTeam']['id']
        # away_team_id = game['awayTeam']['id']
        # home_team = {home_team_id:home_team, away_team_id:away_team}
        self.team = GamesUtils.get_teams(game)


    def draw_event_location(self, game):
        """Plot the location of the current event on the rink image

        Args:
            game (dict): The parsed JSON dictionary containing the game play-by-play informations
        """
        self.rink_output.clear_output(wait=True) #clear the old rink waiting for a new one
        with self.rink_output: 
            game = self.load_game() 
            play = game["plays"][self.event_id_slider.value] 
            details = play.get("details", {}) 
            x = details.get("xCoord") 
            y = details.get("yCoord") 

            if x is not None and y is not None: 
                img = mpimg.imread(self.rink_image_path) 
                height, width, _ = img.shape 
                fig, ax = plt.subplots() 
                image_x = (x + 100) / 200 * width 
                image_y = (42.5 - y) / 85 * height 
                ax.imshow(img) 
                ax.scatter(image_x, image_y, c='red', s=100) 
                ax.axis("off") 
                plt.show()
            else:
                self.rink_output.clear_output(wait=False) #clear the output rink the event have no cordinates
                
    def update_event_id_info(self, game):
            """Update the HTML event info box with the details of the current event

            Args:
                game (dict): The parsed JSON dictionary containing the game play-by-play informations
            """
            play = game["plays"][self.event_id_slider.value]
            event_type = play['typeDescKey']

            if event_type == 'faceoff':
                first_player_id = play['details']['losingPlayerId']
                second_player_id = play['details']['winningPlayerId']
                first_player_name, first_player_face = self.roaster.get(first_player_id)
                second_player_name, second_player_face = self.roaster.get(second_player_id)

                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    Faceoff <br>
                    <img src="{first_player_face}" width="40" style="vertical-align:middle;">
                    {first_player_name} vs {second_player_name}
                    <img src="{second_player_face}" width="40" style="vertical-align:middle;">
                </div>
                """
            elif event_type == 'shot-on-goal':
                first_player_id = play['details']['shootingPlayerId']
                second_player_id = play['details']['goalieInNetId']
                shot_type = play['details']['shotType']
                first_player_name, first_player_face = self.roaster.get(first_player_id)
                second_player_name, second_player_face = self.roaster.get(second_player_id)

                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    Shot on Goal <br>
                    {shot_type} shot <br>
                    <img src="{first_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-right:6px;">
                    {first_player_name}(shooter) on {second_player_name}(goalie)
                    <img src="{second_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-left:6px;">
                </div>"""
            elif event_type == 'blocked-shot':
                first_player_id = play['details']['blockingPlayerId']
                second_player_id = play['details']['shootingPlayerId']
                first_player_name, first_player_face = self.roaster.get(first_player_id)
                second_player_name, second_player_face = self.roaster.get(second_player_id)

                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    Block Shot <br>
                    <img src="{first_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-right:6px;">
                    {first_player_name}(blocker) on {second_player_name}(shooter)
                    <img src="{second_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-left:6px;">
                </div>"""
            elif event_type == 'giveaway':
                first_player_id = play['details']['playerId']
                first_player_name, first_player_face = self.roaster.get(first_player_id)
                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    Giveaway from {first_player_name} <br>
                    <img src="{first_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-right:6px;">
                </div>"""
            elif event_type == 'hit':
                first_player_id = play['details']['hittingPlayerId']
                second_player_id = play['details']['hitteePlayerId']
                first_player_name, first_player_face = self.roaster.get(first_player_id)
                second_player_name, second_player_face = self.roaster.get(second_player_id)

                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    Hit <br>
                    <img src="{first_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-right:6px;">
                    {first_player_name}(blocker) on {second_player_name}(shooter)
                    <img src="{second_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-left:6px;">
                </div>"""
            elif event_type == 'missed-shot':
                first_player_id = play['details']['shootingPlayerId']
                second_player_id = play['details']['goalieInNetId']
                shot_type = play['details']['shotType']
                first_player_name, first_player_face = self.roaster.get(first_player_id)
                second_player_name, second_player_face = self.roaster.get(second_player_id)

                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    Missesd Shot <br>
                    {shot_type} shot <br>
                    <img src="{first_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-right:6px;">
                    {first_player_name}(shooter) on {second_player_name}(goalie)
                    <img src="{second_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-left:6px;">
                </div>"""
            elif event_type == 'takeaway':
                first_player_id = play['details']['playerId']
                first_player_name, first_player_face = self.roaster.get(first_player_id)
                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    Takeaway from {first_player_name} <br>
                    <img src="{first_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-right:6px;">
                </div>"""
            elif event_type == 'stoppage':
                reason = play['details']['reason']
                reason = " ".join(reason.split('-')).capitalize()
                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    Game stop du to {reason} <br>
                </div>"""
            elif event_type == 'penalty':
                first_player_id = play['details']['committedByPlayerId']
                first_player_name, first_player_face = self.roaster.get(first_player_id)
                penalty_type = play['details']['typeCode']
                penalty_reason = play['details']['descKey']
                penalty_duration = play['details']['duration']
                team = self.team.get(play['details']['eventOwnerTeamId'])[0]
                html = f"""        
                <div style="text-align:center; font-weight:bold">
                    {penalty_type} Penalty for {team} <br>
                    <img src="{first_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-right:6px;">
                    {first_player_name} <br>
                    {penalty_duration} minutes for {penalty_reason}
                </div>"""
            elif event_type == 'period-start':
                period_type = play['periodDescriptor']['periodType']
                period_number = play['periodDescriptor']['number']
                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    {period_type} Period {period_number} Start <br>
                </div>"""
            elif event_type == 'period-end':
                period_type = play['periodDescriptor']['periodType']
                period_number = play['periodDescriptor']['number']
                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    {period_type} Period {period_number} End <br>
                </div>"""
            elif event_type == 'goal':
                first_player_id = play['details']['scoringPlayerId']
                second_player_id = play['details']['goalieInNetId']
                shot_type = play['details']['shotType']
                first_player_name, first_player_face = self.roaster.get(first_player_id)
                second_player_name, second_player_face = self.roaster.get(second_player_id)
                team = self.team.get(play['details']['eventOwnerTeamId'])[0]
                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    Goal for {team} <br>
                    {shot_type} shot<br>
                    <img src="{first_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-right:6px;">
                    {first_player_name}(shooter) on {second_player_name}(goalie)
                    <img src="{second_player_face}" width="40" style="vertical-align:middle; border-radius:50%; margin-left:6px;">
                </div>"""
            elif event_type == 'game-end':     
                html = f"""
                <div style="text-align:center; font-weight:bold;">
                    End of Game <br>
                </div>"""
            elif event_type == "delayed-penalty":
                team = self.team.get(play['details']['eventOwnerTeamId'])[0]
                html = f"""        
                <div style="text-align:center; font-weight:bold">
                    Delayed Penalty for {team} <br>
                </div>"""

            else:
                html = f"""        
                <div style="text-align:center; font-weight:bold;">
                    No event
                </div>
                """
            self.event_info_box.value = html

    def load_game(self):
        """Load the JSON file for the current season and game

        Returns:
            dict: The parsed JSON dictionary containing the game play-by-play informations
        """
        season = self.season_drop_down_menu.value
        season_start = season.split('-')[0]   
        game_type = self.game_type.get(self.game_type_radio_button.value)
        season_path = os.path.join(self.data_path, season)
        game_id_num = self.game_id_slider.value
        game_id_num = f"{game_id_num:04d}"
        game_file_name = f"{season_start}{game_type}{game_id_num}.json"
        file = os.path.join(season_path, game_file_name)
        if os.path.exists(file):
            with open(file, 'r') as f:
                game = json.load(f)
        return game
    
    def display(self):
        """Show the widget UI."""
        display(self.ui)

In [3]:
widget = SimpleWidget()
widget.display()

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../games_data'